In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Pernambuco - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_PE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego,Pernambuco - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.297418,34.266080,8.237381e+07,5.574893e+06,8.646979,7.571061e+07,0.688685,8.294029,99.743
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.298912,34.126272,8.248102e+07,5.580364e+06,8.650960,7.575437e+07,0.689009,8.288006,86.997
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.300346,34.414272,8.258823e+07,5.585835e+06,8.654941,7.579812e+07,0.689334,8.281983,83.033
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.301729,34.576094,8.269544e+07,5.591305e+06,8.658922,7.584188e+07,0.689658,8.275960,82.302
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.302368,34.890197,8.280265e+07,5.596776e+06,8.662903,7.588563e+07,0.689983,8.269937,80.217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.478890,26.779678,NaN,NaN,NaN,NaN,NaN,NaN,164.137
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.478683,26.996501,NaN,NaN,NaN,NaN,NaN,NaN,168.356
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.476012,26.919241,NaN,NaN,NaN,NaN,NaN,NaN,172.421
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.473486,27.026250,NaN,NaN,NaN,NaN,NaN,NaN,165.411


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.011454,-1.188910,-1.754154,-0.181030,-2.511326,-2.321118,-2.100972,-0.864836
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.964614,-1.195377,-1.734705,-0.148206,-2.462275,-2.280399,-2.067858,-0.866889
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.919667,-1.182056,-1.715257,-0.115382,-2.413223,-2.239680,-2.034743,-0.868942
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.876288,-1.174571,-1.695809,-0.082559,-2.364172,-2.198961,-2.001628,-0.870995
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.856283,-1.160042,-1.676361,-0.049735,-2.315120,-2.158242,-1.968514,-0.873048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.720800,-1.281757,1.016740,-1.416488,-0.070259,0.355961,1.324101,1.183158
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.802101,-1.271596,1.004554,-1.402870,-0.093306,0.336941,1.314733,1.181904
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.813469,-1.268136,0.992368,-1.389252,-0.116354,0.317922,1.305366,1.180650
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.843043,-1.270527,0.980182,-1.375634,-0.139402,0.298902,1.295999,1.179397


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      76.341
1      65.040
2      88.306
3      74.323
4      78.114
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Pernambuco - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.011454,-1.188910,-1.754154,-0.181030,-2.511326,-2.321118,-2.100972,-0.864836
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.964614,-1.195377,-1.734705,-0.148206,-2.462275,-2.280399,-2.067858,-0.866889
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.919667,-1.182056,-1.715257,-0.115382,-2.413223,-2.239680,-2.034743,-0.868942
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.876288,-1.174571,-1.695809,-0.082559,-2.364172,-2.198961,-2.001628,-0.870995
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.856283,-1.160042,-1.676361,-0.049735,-2.315120,-2.158242,-1.968514,-0.873048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.354413,-0.165538,1.170965,-1.488331,0.445555,0.786479,1.391411,1.357017
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.374652,-0.236996,1.172520,-1.494202,0.438851,0.781348,1.390568,1.349817
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.400582,-0.309267,1.174075,-1.500073,0.432147,0.776217,1.389726,1.342617
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.422300,-0.382346,1.175630,-1.505943,0.425442,0.771086,1.388883,1.335417


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       76.341
1       65.040
2       88.306
3       74.323
4       78.114
        ...   
157    143.388
158    188.227
159    143.049
160    148.068
161    133.872
Name: Pernambuco - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,1.276351,1.507092,0.863757,-0.284837,0.991047,1.042288,0.483520,0.905318
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,1.291616,1.497914,0.879821,-0.344037,0.974598,1.037752,0.491009,0.926343
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.299215,1.471401,0.895884,-0.403236,0.958148,1.033215,0.498499,0.947367
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.301690,1.429249,0.911948,-0.462436,0.941699,1.028678,0.505988,0.968392
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.304954,1.390164,0.928011,-0.521635,0.925249,1.024142,0.513478,0.989417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.720800,-1.281757,1.016740,-1.416488,-0.070259,0.355961,1.324101,1.183158
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.802101,-1.271596,1.004554,-1.402870,-0.093306,0.336941,1.314733,1.181904
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.813469,-1.268136,0.992368,-1.389252,-0.116354,0.317922,1.305366,1.180650
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.843043,-1.270527,0.980182,-1.375634,-0.139402,0.298902,1.295999,1.179397


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=True), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2634021302, 134995750, 135212944, 2660622104, 2572114934, 803990699, 2356045751, 2270762597, 1531017573, 1275953393, 2964696399, 3944319602, 3774538040, 506923291, 1322509873, 3601426681, 2938869118, 1712816024, 614093169, 3245149834, 227653472, 332189712, 382452, 2447576188, 1845773320, 1314312913, 1825144302, 1934104088, 874834366, 3333916952, 763757867, 1107519684, 3505290202, 2163454857, 231299549, 3307392452, 955324584, 546282442, 1719502041, 326558425, 1205062172, 3007748629, 2619100457, 3946336875, 1730856644, 2613802536, 3173664040, 2454540846, 1417397934, 560708663]


Step: 0 ___________________________________________
val_loss: 165.0691680908203
winner_seed: 2634021302


Step: 1 ___________________________________________
val_loss: 175.62916564941406


Step: 2 ___________________________________________
val_loss: 218.3202362060547


Step: 3 ___________________________________________
val_loss: 196.95350646972656


Step: 4 ___________________________________________
val_loss:

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 4s 444ms/step - loss: 28848.8867 - val_loss: 40574.2227
Epoch 2/10000
4/4 [==============================] - 0s 21ms/step - loss: 93311.1875 - val_loss: 138128.3906
Epoch 3/10000
4/4 [==============================] - 0s 20ms/step - loss: 52338.6406 - val_loss: 1552.6774
Epoch 4/10000
4/4 [==============================] - 0s 19ms/step - loss: 2330.2673 - val_loss: 2155.1938
Epoch 5/10000
4/4 [==============================] - 0s 21ms/step - loss: 1817.2307 - val_loss: 2093.1379
Epoch 6/10000
4/4 [==============================] - 0s 16ms/step - loss: 1620.0372 - val_loss: 2222.0205
Epoch 7/10000
4/4 [==============================] - 0s 20ms/step - loss: 1708.0896 - val_loss: 1384.8477
Epoch 8/10000
4/4 [==============================] - 0s 18ms/step - loss: 1073.9598 - val_loss: 1013.9626
Epoch 9/10000
4/4 [==============================] - 0s 17ms/step - loss: 1196.9481 - val_loss: 1205.9935
Epoch 10/10000
4/4 [===================

4/4 [==============================] - 0s 19ms/step - loss: 444.5864 - val_loss: 433.4797
Epoch 79/10000
4/4 [==============================] - 0s 17ms/step - loss: 404.7112 - val_loss: 263.5390
Epoch 80/10000
4/4 [==============================] - 0s 18ms/step - loss: 369.6208 - val_loss: 281.9694
Epoch 81/10000
4/4 [==============================] - 0s 19ms/step - loss: 318.5277 - val_loss: 375.6389
Epoch 82/10000
4/4 [==============================] - 0s 17ms/step - loss: 319.0451 - val_loss: 223.3764
Epoch 83/10000
4/4 [==============================] - 0s 19ms/step - loss: 378.2674 - val_loss: 283.7665
Epoch 84/10000
4/4 [==============================] - 0s 17ms/step - loss: 331.2680 - val_loss: 187.0163
Epoch 85/10000
4/4 [==============================] - 0s 18ms/step - loss: 355.2269 - val_loss: 431.7816
Epoch 86/10000
4/4 [==============================] - 0s 19ms/step - loss: 308.7054 - val_loss: 195.5551
Epoch 87/10000
4/4 [==============================] - 0s 19ms/step - l

4/4 [==============================] - 0s 21ms/step - loss: 330.2780 - val_loss: 192.4887
Epoch 156/10000
4/4 [==============================] - 0s 21ms/step - loss: 266.9058 - val_loss: 267.5378
Epoch 157/10000
4/4 [==============================] - 0s 20ms/step - loss: 258.9229 - val_loss: 263.0724
Epoch 158/10000
4/4 [==============================] - 0s 19ms/step - loss: 257.2055 - val_loss: 261.3839
Epoch 159/10000
4/4 [==============================] - 0s 19ms/step - loss: 396.9098 - val_loss: 251.0536
Epoch 160/10000
4/4 [==============================] - 0s 18ms/step - loss: 319.1904 - val_loss: 233.0644
Epoch 161/10000
4/4 [==============================] - 0s 15ms/step - loss: 360.0538 - val_loss: 245.0202
Epoch 162/10000
4/4 [==============================] - 0s 17ms/step - loss: 280.9757 - val_loss: 230.1432
Epoch 163/10000
4/4 [==============================] - 0s 17ms/step - loss: 280.0298 - val_loss: 217.8775
Epoch 164/10000
4/4 [==============================] - 0s 17ms

4/4 [==============================] - 0s 18ms/step - loss: 380.6409 - val_loss: 579.9443
Epoch 233/10000
4/4 [==============================] - 0s 18ms/step - loss: 260.7104 - val_loss: 512.0840
Epoch 234/10000
4/4 [==============================] - 0s 20ms/step - loss: 280.3879 - val_loss: 293.2457
Epoch 235/10000
4/4 [==============================] - 0s 20ms/step - loss: 305.5963 - val_loss: 274.9689
Epoch 236/10000
4/4 [==============================] - 0s 16ms/step - loss: 273.6272 - val_loss: 240.5326
Epoch 237/10000
4/4 [==============================] - 0s 19ms/step - loss: 334.5464 - val_loss: 442.9005
Epoch 238/10000
4/4 [==============================] - 0s 18ms/step - loss: 298.2908 - val_loss: 242.8321
Epoch 239/10000
4/4 [==============================] - 0s 18ms/step - loss: 261.8407 - val_loss: 245.4998
Epoch 240/10000
4/4 [==============================] - 0s 16ms/step - loss: 268.3946 - val_loss: 277.5593
Epoch 241/10000
4/4 [==============================] - 0s 18ms

4/4 [==============================] - 0s 17ms/step - loss: 299.9420 - val_loss: 306.5363
Epoch 310/10000
4/4 [==============================] - 0s 20ms/step - loss: 222.0159 - val_loss: 529.6712
Epoch 311/10000
4/4 [==============================] - 0s 19ms/step - loss: 251.0026 - val_loss: 378.7730
Epoch 312/10000
4/4 [==============================] - 0s 16ms/step - loss: 229.9265 - val_loss: 236.2080
Epoch 313/10000
4/4 [==============================] - 0s 17ms/step - loss: 320.1338 - val_loss: 189.2893
Epoch 314/10000
4/4 [==============================] - 0s 16ms/step - loss: 278.8853 - val_loss: 186.3721
Epoch 315/10000
4/4 [==============================] - 0s 17ms/step - loss: 325.8477 - val_loss: 418.6388
Epoch 316/10000
4/4 [==============================] - 0s 18ms/step - loss: 295.9243 - val_loss: 206.5547
Epoch 317/10000
4/4 [==============================] - 0s 17ms/step - loss: 286.5038 - val_loss: 187.1805
Epoch 318/10000
4/4 [==============================] - 0s 17ms

...

4/4 [==============================] - 0s 12ms/step - loss: 148.2318 - val_loss: 153.3221
Epoch 1231/10000
4/4 [==============================] - 0s 14ms/step - loss: 147.3740 - val_loss: 172.1720
Epoch 1232/10000
4/4 [==============================] - 0s 14ms/step - loss: 151.4580 - val_loss: 130.6650
Epoch 1233/10000
4/4 [==============================] - 0s 15ms/step - loss: 180.4915 - val_loss: 122.6991
Epoch 1234/10000
4/4 [==============================] - 0s 16ms/step - loss: 153.3148 - val_loss: 142.6215
Epoch 1235/10000
4/4 [==============================] - 0s 16ms/step - loss: 135.6595 - val_loss: 213.6223
Epoch 1236/10000
4/4 [==============================] - 0s 16ms/step - loss: 162.9613 - val_loss: 256.0015
Epoch 1237/10000
4/4 [==============================] - 0s 15ms/step - loss: 161.7514 - val_loss: 182.8244
Epoch 1238/10000
4/4 [==============================] - 0s 18ms/step - loss: 121.3405 - val_loss: 135.2564
Epoch 1239/10000
4/4 [==============================] 

4/4 [==============================] - 0s 18ms/step - loss: 141.0933 - val_loss: 169.5219
Epoch 1307/10000
4/4 [==============================] - 0s 19ms/step - loss: 201.8517 - val_loss: 153.5892
Epoch 1308/10000
4/4 [==============================] - 0s 19ms/step - loss: 119.3640 - val_loss: 205.4727
Epoch 1309/10000
4/4 [==============================] - 0s 19ms/step - loss: 150.2918 - val_loss: 164.7774
Epoch 1310/10000
4/4 [==============================] - 0s 19ms/step - loss: 111.1429 - val_loss: 154.3029
Epoch 1311/10000
4/4 [==============================] - 0s 17ms/step - loss: 152.5639 - val_loss: 175.5271
Epoch 1312/10000
4/4 [==============================] - 0s 18ms/step - loss: 131.9465 - val_loss: 279.3665
Epoch 1313/10000
4/4 [==============================] - 0s 17ms/step - loss: 226.8434 - val_loss: 379.7973
Epoch 1314/10000
4/4 [==============================] - 0s 18ms/step - loss: 148.2817 - val_loss: 191.9789
Epoch 1315/10000
4/4 [==============================] 

4/4 [==============================] - 0s 17ms/step - loss: 141.7755 - val_loss: 255.8118
Epoch 1383/10000
4/4 [==============================] - 0s 18ms/step - loss: 123.2113 - val_loss: 422.3316
Epoch 1384/10000
4/4 [==============================] - 0s 17ms/step - loss: 178.7865 - val_loss: 233.2375
Epoch 1385/10000
4/4 [==============================] - 0s 17ms/step - loss: 159.7901 - val_loss: 232.3831
Epoch 1386/10000
4/4 [==============================] - 0s 18ms/step - loss: 104.5732 - val_loss: 192.5766
Epoch 1387/10000
4/4 [==============================] - 0s 19ms/step - loss: 164.1787 - val_loss: 216.0564
Epoch 1388/10000
4/4 [==============================] - 0s 19ms/step - loss: 114.2551 - val_loss: 214.7691
Epoch 1389/10000
4/4 [==============================] - 0s 19ms/step - loss: 133.2899 - val_loss: 236.8362
Epoch 1390/10000
4/4 [==============================] - 0s 17ms/step - loss: 135.9348 - val_loss: 192.1390
Epoch 1391/10000
4/4 [==============================] 

4/4 [==============================] - 0s 18ms/step - loss: 118.9626 - val_loss: 158.3575
Epoch 1459/10000
4/4 [==============================] - 0s 18ms/step - loss: 149.7445 - val_loss: 226.8156
Epoch 1460/10000
4/4 [==============================] - 0s 18ms/step - loss: 174.1452 - val_loss: 137.4372
Epoch 1461/10000
4/4 [==============================] - 0s 18ms/step - loss: 130.2450 - val_loss: 232.0513
Epoch 1462/10000
4/4 [==============================] - 0s 19ms/step - loss: 178.9028 - val_loss: 510.2889
Epoch 1463/10000
4/4 [==============================] - 0s 20ms/step - loss: 429.7273 - val_loss: 317.2341
Epoch 1464/10000
4/4 [==============================] - 0s 17ms/step - loss: 245.7903 - val_loss: 245.6060
Epoch 1465/10000
4/4 [==============================] - 0s 16ms/step - loss: 272.2012 - val_loss: 274.5923
Epoch 1466/10000
4/4 [==============================] - 0s 20ms/step - loss: 185.9066 - val_loss: 291.2220
Epoch 1467/10000
4/4 [==============================] 

4/4 [==============================] - 0s 18ms/step - loss: 349.7084 - val_loss: 306.6087
Epoch 1535/10000
4/4 [==============================] - 0s 18ms/step - loss: 181.8749 - val_loss: 335.8483
Epoch 1536/10000
4/4 [==============================] - 0s 17ms/step - loss: 201.7508 - val_loss: 289.4250
Epoch 1537/10000
4/4 [==============================] - 0s 17ms/step - loss: 218.7086 - val_loss: 356.3192
Epoch 1538/10000
4/4 [==============================] - 0s 18ms/step - loss: 360.4330 - val_loss: 346.5140
Epoch 1539/10000
4/4 [==============================] - 0s 18ms/step - loss: 236.8407 - val_loss: 344.9691
Epoch 1540/10000
4/4 [==============================] - 0s 17ms/step - loss: 191.2654 - val_loss: 321.9044
Epoch 1541/10000
4/4 [==============================] - 0s 19ms/step - loss: 204.8661 - val_loss: 297.8474
Epoch 1542/10000
4/4 [==============================] - 0s 19ms/step - loss: 222.4441 - val_loss: 668.3089
Epoch 1543/10000
4/4 [==============================] 

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,146.093735,143.079132,145.219177,141.307373,142.897308,143.009064,143.176483,140.908875,140.504547,140.793411,140.031784,145.653748,139.65387,144.791916,142.056534,143.917892,141.949493,142.908478,146.959396,143.940186,146.348938,143.776154,142.763855,144.557526,143.828857,144.376572,143.019836,144.440872,143.328873,143.912567
Target,131.464,151.37,154.276,153.331,152.6,136.567,144.855,126.352,138.148,129.978,109.083,147.44,132.5,158.533,133.813,155.827,146.197,122.278,149.983,123.483,125.104,129.633,138.233,109.657,128.809,140.408,134.634,189.04,166.406,155.456
Error,14.62973,8.290863,9.056824,12.023621,9.702698,6.442062,1.678513,14.556877,2.356552,10.815414,30.948784,1.786255,7.15387,13.741089,8.24353,11.909103,4.247513,20.630478,3.023605,20.457184,21.244942,14.143158,4.530853,34.900528,15.019852,3.968567,8.385834,44.599121,23.077133,11.543427


In [38]:
display(mae)
display(mape)

13.103598

0.09617022

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 17ms/step
Ano-0: |Prediction[[1712.6746]] - Target[1675.4640000000002]| =  Error: [[37.21057]]; MAPE:[[0.02220911]]
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[1723.6241]] - Target[1625.241]| =  Error: [[98.38318]]; MAPE:[[0.06053452]]
1/1 [==============================] - 0s 17ms/step
Ano-5: |Prediction[[862.9076]] - Target[914.753]| =  Error: [[51.845398]]; MAPE:[[0.05667694]]


[array([[37.21057]], dtype=float32),
 array([[98.38318]], dtype=float32),
 array([[51.845398]], dtype=float32)]

62.479717

0.04647352